In [1]:
from main_library import *

################################
## data load / adjustements*
extractDate = date_load()

### pour l'instant ne fonctionne pas
## demander à Eric de relancer la machine sur sandbox
# get_call_info()

# get_call_info_europa()

proj = projects_load()
proj_id_signed = proj.project_id.unique()

prop = proposals_load()
proj = proj_add_cols(prop, proj)

stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
prop1 = proposals_status(prop, proj_id_signed, stage_p)  
# np.save("data_files/applicants_columns.npy", prop_cols)

###########################################
# proposals fix
# projects missing from proposals
call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# project data missing in proposals if call already in proposals then add this
proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
call_miss = list(set(call_miss)-set(call_to_integrate))
proj = proj.loc[~proj.callId.isin(call_miss)]

# merge proj + prop
print('### MERGED PROPOSALS/PROJECTS')
if len(proj1)==0:
    prop2=pd.concat([proj,prop1], ignore_index= True)
else:
    prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")

merged = copy.deepcopy(prop2)
merged = dates_year(merged)
merged = strings_v(merged)
merged = url_to_clean(merged)
merged.mask(merged=='', inplace=True)
merged = empty_str_to_none(merged)      
merged.rename(columns={
    'freekw':'free_keywords',
    'callDeadlineDate':'call_deadline', 
    'callId':'call_id', 
    'submissionDate':'submission_date',
    'startDate':'start_date',
    'endDate':'end_date', 
    'ecSignatureDate':'signature_date'}, inplace=True)

if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
    print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

merged = merged_panels(merged)
merged = merged_topics(merged)
merged = merged_actions(merged)

# calls list
calls = call(PATH_SOURCE+FRAMEWORK+'/')

print("\n### CALLS+MERGED")
if len(merged.loc[merged.call_id.isnull()])>0:
        print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
else:
    call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
    print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

calls = calls_to_check(calls, call_id)

projects = projects_complete_cleaned(merged, extractDate)

#############################################################
##### PARTICIPATIONS
part = participants_load(proj)
# conserve uniquement les projets présents dans proposals et applicants
part = part.loc[part.project_id.isin(projects.project_id.unique())]
print(f"- size part hors proj manquant: {len(part)}")
part = part_role_type(part)
part = erc_role(part, projects)

#### APPLICANTS
app = applicants_load(prop)
# conserve uniquement les projets présents dans proposals et applicants
app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
print(f"- size app1 hors proj exclus: {len(app1)}")

app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
tmp = part[part.project_id.isin(app_missing_pid)]
app1 = part_miss_app(tmp, app1)

#redressement accelerator
acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
print(f"size acc_folio: {len(acc_folio)}")
acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
       .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
       .drop(columns='PROPOSAL_NBR'))
print(f"size acc: {len(acc)}")
app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

app1 = app_role_type(app1)
app1 = erc_role(app1, projects)

del app

####
# verification Etat des participations
part = check_multiP_by_proj(part)
app1 = check_multiA_by_proj(app1)


### STEP2
lien = merged_partApp(app1, part)
lien = nuts_lien(app1, part, lien)
lien.to_pickle(f"{PATH_CLEAN}lien.pkl")

# ENTITIES
entities = entities_load(lien)

list_codeCountry = list(set(entities.countryCode.to_list()+lien.countryCode.to_list()))
countries, countryCode_err = country_load(FRAMEWORK, list_codeCountry)
if countryCode_err:
    ccode=json.load(open("data_files/countryCode_match.json"))
    for i in ccode:
        for k,v in i.items():
            lien.loc[lien.countryCode==k, 'countryCode'] = v
            entities.loc[entities.countryCode==k, 'countryCode'] = v

#ENTITIES +LIEN
entities_single = entities_single_create(entities, lien)
entities_info = entities_info_create(entities_single, lien)

### LAST DATE of EXTRACTED DATA
[{'extraction_date': '2024-10-09', 'framework': 'HORIZON'}]

### LOADING PROJECTS data
- no new columns
- result -> dowloaded projects:17382, retained projects:17382, pb:0
- liste des colonnes conservées:
Index(['project_id', 'callId', 'callDeadlineDate', 'typeOfActionCode',
       'acronym', 'status_code', 'startDate', 'endDate', 'ecSignatureDate',
       'title', 'abstract', 'url', 'topicCode', 'duration', 'total_cost',
       'eu_reqrec_grant', 'number_involved', 'framework', 'lastUpdateDate',
       'totalGrant', 'nationalContribution', 'otherContribution', 'freekw',
       'stage'],
      dtype='object')

### LOADING PROPOSALS data
- no new columns
1- empty cols -> Attention ! vérifier les variables manquantes->[]
- result -> dowloaded proposals:92864, retained proposals:92864, pb:0

### ADD COLS TO PROJECTS FROM PROPOSALS
- liste des variables PROJ en plus: ['totalGrant', 'nationalContribution', 'ecSignatureDate', 'otherContribution', 'endDate', 'u

c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


- code panel existant pour d'autres programmes que erc/msca: données exportées

- size merged after add panels: 102295
### TOPICS
1 - topics -> 2448
2 - divisions -> 3172
erc : destination_code à null après traitement
['HORIZON-ERC-2021-VICECHAIRS-IBA' 'ERC-2025-NCPS-IBA'
 'HORIZON-ERC-2021-arXiv-IBA'
 'HORIZON-CL4-2024-SSA-SST-SD-IBA-to-be-deleted'
 'HORIZON-CL4-2024-SSA-SST-AE-IBA-to-be-deleted'
 'HORIZON-ERC-2023-VICECHAIRS-IBA' 'HORIZON-ERC-2022-VICECHAIRS-IBA'
 'HORIZON-CL4-2024-SSA-SST-SP-IBA-to-be-deleted']
MSCA : destination_code à null après traitement
['HORIZON-MSCA-2021-INCO-01-01' 'HORIZON-MSCA-2021-FRC-IBA'
 'HORIZON-MSCA-2022-ALUMNI-IBA' 'HORIZON-MSCA-2024-NCP-01-01'
 'HORIZON-MSCA-2023-FTP-01-01' 'HORIZON-MSCA-2022-Ukraine-ART195-IBA'
 'HORIZON-MSCA-2021-RR-01-01' 'HORIZON-MSCA-2024-RR-01-01'
 'HORIZON-MSCA-2023-BEL-IBA' 'HORIZON-MSCA-2024-UKRAINE-IBA'
 'HORIZON-MSCA-2024-INCO-01-01' 'HORIZON-MSCA-SNLS-2021-IBA'
 'HORIZON-MSCA-2021-NCP-01-01' 'HORIZON-MSCA-2024-ALUMNI-IB

c:\Users\zfriant\Documents\GitHub\pcri\step1_mainData\topics.py:171: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SESAR' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top.loc[mask, 'destination_code'] = i


size merged after add topics: 102295

### ACTIONS
1- call sans action:
                                      call_id
24693  HORIZON-HLTH-2022-DISEASE-06-two-stage

- size merged after add actions: 102295


### CALLS

### CALLS+MERGED
2 - CALL_ID de merged -> nb call+deadline: 530, nb call unique: 492 

### CALLS to CHECK
- CALLS de base calls with merge call_id ->
nb call+deadline: 537, nb call unique: 492 
2- si calls pas dans call_id :
                             call_id call_deadline
231  HORIZON-EIC-2023-BOOSTER-IBA-01    2023-07-05
248  HORIZON-EIC-2024-ACCELERATOR-02    2024-10-03
251  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-09-04
252  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-11-14
467       HORIZON-MSCA-2021-SNLS-IBA    2021-07-27
505    HORIZON-WIDERA-2022-ACCESS-07    2022-04-20
522    HORIZON-WIDERA-2023-ACCESS-06    2024-09-26
3- calls with multi deadline
                                     call_id call_deadline   call_budget
17                              ERC-2023-POC    

In [6]:
ref_source = ref_source_load('ref')
ref, genPic_to_new = ref_source_2d_select(ref_source, 'HE')
entities_tmp = entities_tmp_create(entities_info, countries, ref)
print(f"size entities_tmp: {len(entities_tmp)}")
entities_tmp = entities_for_merge(entities_tmp)


### LOADING REF_SOURCE
- size of ref_source : 503464
### 2d - REF_SOURCE -> REF
- size remplacement pic: 445
- longueur de ref:79070
- nb id: 79067
- nb id after fill: 79067
### create ENTITIES TMP pour ref
- size entities_info before:77984, size entities_info+ref -> tmp:77963, Pic unique tmp:77898
# missing entities into ref
      generalPic countryCode                         legalName  \
19625  886944430          UK       PLanetWatchers (UK) Limited   
22604  887969235          UK               Botanico Design Ltd   
23676  888299423          UK   Eternum Ltd T/A Solaris Offgrid   
28097  889536367          UK               3ti Energy Hubs Ltd   
28638  889741910          EL  Tsergoulas K. - Tsergoulas E. OE   
28738  889782165          UK  J E Piccaver & Co (Gedney Marsh)   
28829  889819025          UK                  Petalite Limited   
29104  889937268          UK  Advanced Agri-Tech Solutions Ltd   
29131  889947162          UK                  Mustafa Khraishi   
29252  8899

In [13]:
# ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
# entities_tmp = merge_ror(entities_tmp, ror)

# PAYSAGE
### si besoin de charger paysage pickle
paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
# paysage_category = IDpaysage_category(paysage)
cat_filter = category_paysage(paysage_category)
entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)

# SIRENE
### si besoin de charger paysage pickle
sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
entities_tmp = merge_sirene(entities_tmp, sirene)

entities_tmp.loc[(~entities_tmp.id.isnull())&(entities_tmp.entities_id.isnull()), 'entities_id'] = entities_tmp.id

if any(entities_tmp.siren.str.contains(';', na=False)):
    print("ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")

### CATEGORY paysage
### merge PAYSAGE
- doublons PIC      generalPic                                          legalName  \
119    876941596                       WEIR MINERALS EUROPE LIMITED   
120    876941596                       WEIR MINERALS EUROPE LIMITED   
173    877256167                                  EURACTIV MEDIA BV   
174    877256167                             EURACTIV MEDIA NETWORK   
320    877625834  NEPTUNE LINES SHIPPING AND MANAGING ENTERPRISE...   
...          ...                                                ...   
77941  999988230              EUROPEAN MOLECULAR BIOLOGY LABORATORY   
77942  999988230              EUROPEAN MOLECULAR BIOLOGY LABORATORY   
77943  999988230              EUROPEAN MOLECULAR BIOLOGY LABORATORY   
77954  999990267  MAX-PLANCK-GESELLSCHAFT ZUR FORDERUNG DER WISS...   
77955  999990267  MAX-PLANCK-GESELLSCHAFT ZUR FORDERUNG DER WISS...   

        businessName          id id_secondaire ZONAGE country_code_mapping  \
119    WEIR MINE

In [14]:
entities_tmp[entities_tmp.generalPic=='886944430']
# ref[ref.generalPic=='886944430'].merge(countries[['countryCode', 'country_code_mapping', 'country_name_mapping', 'countryCode_parent']], how='left', on='countryCode')

,generalPic,legalName,businessName,id,id_secondaire,ZONAGE,country_code_mapping,countryCode_parent,id_extend,unused_parent,...,siren_end_date,paysage_siren,nb,paysage_category_id,paysage_category,paysage_category_priority,siret_closeDate,id_m,siren,siege
0,886944430,PLanetWatchers (UK) Limited,PlanetWatchers,NaN,NaN,NaN,GBR,UK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
import pandas as pd
def IDpic(entities_tmp):
    print("\n### traitement id_pic avec tiret")
    # IDENT with '-' : traitement des identifiants avec '-' pour regrouper multi-pic non identifiés 
    if not entities_tmp.loc[entities_tmp.id.str.contains('-', na=False)].empty:
        pic_dash = (entities_tmp.loc[entities_tmp.id.str.contains('-', na=False), ['generalPic', 'entities_id']]
        .drop_duplicates()
        .drop(columns='generalPic')
        .assign(pic_d = entities_tmp.entities_id.str.split('-').str[0]))
        print(f"- size entities pic_dash: {len(pic_dash)}")
        dash = (pic_dash.merge(entities_tmp, how='inner', left_on='pic_d', right_on='generalPic', suffixes=['_x',''])
                .drop(columns=['entities_id_x', 'pic_d'])).drop_duplicates()
        
        entities_tmp = entities_tmp.loc[~entities_tmp.entities_id.isin(dash.entities_id.unique())]
        entities_tmp = pd.concat([entities_tmp, dash], ignore_index=True)
        print(f"- size entities_tmp: {len(entities_tmp)}")

    # IDENT pic : corriger appliquer les lignes ci-dessous uniuqument sur entities_id est null ou commence par pic
    entities_tmp.loc[entities_tmp.entities_id.isnull(), 'entities_id'] = "pic"+entities_tmp.generalPic.map(str)
    print(f"- End size entities_tmp: {len(entities_tmp)}")
    return entities_tmp
entities_tmp = IDpic(entities_tmp)


### traitement id_pic avec tiret
- End size entities_tmp: 77988


In [17]:
entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')

### sourcer les identifiants pour getInformations


In [19]:

### si besoin de charger groupe 
groupe = pd.read_pickle(f"{PATH_REF}groupe.pkl")
print(f"taille de entities_tmp avant groupe:{len(entities_tmp)}")
entities_tmp = merge_groupe(entities_tmp, groupe)

taille de entities_tmp avant groupe:77988

### merge avec GROUPE
taille de entities_tmp après groupe 77988


In [ ]:

print("### ENTITIES TMP cleaning name")
x=entities_tmp[entities_tmp.entities_name.isnull()]
print(f"- size entities_tmp without entities_name: {len(x)}")
if not x.empty:
    if any(x.loc[(x.legalName.str.contains("\\00",  na=True))]):
        print(x.loc[(x.legalName.str.contains("\\00",  na=True))].legalName)

    y=x.loc[(x.businessName.str.contains("00",  na=True))]
    for i, row in y.iterrows():
        try:
            y.at[i, 'businessName'] = row.businessName.replace("\\", "\\u").encode().decode('unicode_escape')
        except:
            y.at[i, 'businessName'] = np.nan
    x = x.loc[~x.generalPic.isin(y.generalPic.unique())]
    x = pd.concat([x, y], ignore_index=True)

    x.loc[x.businessName.str.contains('^\\d+$', na=True), 'businessName'] = np.nan
    liste=['legalName', 'businessName']
    for i in liste:
        x[i] = x[i].apply(lambda v: v.capitalize().strip() if isinstance(v, str) else v)

    print(f"- End size entities_tmp without entities_name: {len(x)}")
    
    entities_tmp = entities_tmp.loc[~entities_tmp.generalPic.isin(x.generalPic.unique())]
    entities_tmp = pd.concat([entities_tmp, x], ignore_index=True)

    entities_tmp.loc[entities_tmp.entities_name.isnull(), 'entities_acronym'] = entities_tmp.loc[entities_tmp.entities_name.isnull()].businessName
    entities_tmp.loc[entities_tmp.entities_name.isnull(), 'entities_name'] = entities_tmp.loc[entities_tmp.entities_name.isnull()].legalName
    print(f"- size entities_tmp: {len(entities_tmp)}")

### ENTITIES TMP cleaning name
Series([], Name: legalName, dtype: object)
- size entities_tmp: 77987


In [ ]:
liste=['entities_name_source', 'entities_acronym_source']
for i in liste:
    entities_tmp[i] = entities_tmp[i].apply(lambda x: x.capitalize().strip() if isinstance(x, str) else x)
    
entities_tmp.loc[entities_tmp.entities_name.isnull(), 'entities_name'] = entities_tmp.entities_name_source
entities_tmp.loc[(entities_tmp.id.isnull())&(entities_tmp.entities_acronym.isnull()), 'entities_acronym'] = entities_tmp.entities_acronym_source



,generalPic,legalName,businessName,id,id_secondaire,ZONAGE,country_code_mapping,countryCode_parent,id_extend,unused_parent,...,paysage_category_priority,siret_closeDate,id_m,siren,siege,source_id,groupe_name,groupe_acronym,groupe_id,groupe_sector
0,886944430,PLanetWatchers (UK) Limited,PlanetWatchers,NaN,NaN,NaN,GBR,UK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
1,887969235,Botanico Design Ltd,Canopi,NaN,NaN,NaN,GBR,UK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
2,889741910,Tsergoulas K. - Tsergoulas E. OE,Tsergoulas K. - Tsergoulas E. OE,NaN,NaN,NaN,GRC,EL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
3,889819025,Petalite Limited,Petalite Limited,NaN,NaN,NaN,GBR,UK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
4,889937268,Advanced Agri-Tech Solutions Ltd,AATS,NaN,NaN,NaN,GBR,UK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,999952922,MALOPOLSKA VOIVODSHIP,NaN,NaN,NaN,NaN,POL,PL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
59267,999954959,SOCIETE ANONYME BELGE DE CONSTRUCTIONS AERONAU...,NaN,NaN,NaN,NaN,BEL,BE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
59268,999956317,HYDROGEN EUROPE,NaN,NaN,NaN,NaN,BEL,BE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN
59269,999966502,BAVAIRIA EV,NaN,NaN,NaN,NaN,DEU,DE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pic,NaN,NaN,NaN,NaN


In [27]:
# entities_tmp[entities_tmp.generalPic=='999954183']
# entities_tmp[(entities_tmp.legalName.str.contains('rte', case=False))&(entities_tmp.country_code_mapping=='FRA')]
entities_tmp.entities_name

0        Association francaise pour la prevention des c...
1                                                Ecomaison
2                Communaute d'agglomeration de lens-lievin
3                       Office for climate education (oce)
4                              Chambery-grand lac economie
                               ...                        
77982                                MALOPOLSKA VOIVODSHIP
77983    SOCIETE ANONYME BELGE DE CONSTRUCTIONS AERONAU...
77984                                      HYDROGEN EUROPE
77985                                          BAVAIRIA EV
77986                                 REINZ-DICHTUNGS GMBH
Name: entities_name, Length: 77987, dtype: object